# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
! pip install -U langchain

In [2]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [3]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [4]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [5]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

C:\Users\lenovo\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 第几届大运会是在哪一年举办的？
Intermediate answer: 2021年夏季世界大学生运动会（英语：2021 Summer World University Games）简称为2021年成都大运会或成都大运会，是第31届夏季世界大学生运动会，原定于2021年于中国四川省成都市举行，后因2019冠状病毒病疫情等原因二次延期，最终改至2023年7月28日至8月8日举行。
Follow up: 2023年大运会的举办地是哪里？
Intermediate answer: 2018年3月5日正式宣布普莱西德湖成为2023年冬季大运会的主办城市，国际大体联官员称这是大运会申办历史上最强有力的一次。 由于2021年冬季世界大学生运动会因2019冠状病毒病疫情而取消，将大运会会旗正式移交给普莱西德湖的仪式改在2022年1月于都灵举行。
So the final answer is: 第31届大运会，2023年，普莱西德湖。

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '第31届大运会，2023年，普莱西德湖。'}

In [7]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 2023年大运会是第几届大运会？
Intermediate answer: 2023成都第31届世界大学生运动会开幕式& 2023 The 31 FISU World University Games - Opening Ceremony(4K HDR) - YouTube.
Follow up: 成都是第几届大运会举办地？
Intermediate answer: 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施，是规模仅次于奥运会的世界综合性运动会，素有“小奥运会”之称。 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
So the final answer is: 成都

> Finished chain.


{'input': '2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？', 'output': '成都'}

In [8]:
# Reason-only 答案（错误）
print(llm.invoke("成都举办的大运会是第几届大运会？"))



成都举办的大运会是第31届大运会。


In [9]:
# Reason-only 答案（错误）
print(llm.invoke("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [16]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [18]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.invoke(
     "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.

Follow up: 成都举办的大运会是哪一年？
Intermediate answer: 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施，是规模仅次于奥运会的世界综合性运动会，素有“小奥运会”之称。 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
Intermediate answer: 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。

So the final answer is: 第31届

Follow up: 2023年大运会举办地在哪里？
Intermediate answer: 2018年3月5日正式宣布普莱西德湖成为2023年冬季大运会的主办城市，国际大体联官员称这是大运会申办历史上最强有力的一次。 由于2021年冬季世界大学生运动会因2019冠状病毒病疫情而取消，将大运会会旗正式移交给普莱西德湖的仪式改在2022年1月于都灵举行。
Intermediate answer: 2023年冬季大运会的举办地是普莱西德湖。

So the final answer is: 普莱西德湖

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '普莱西德湖'}